# Домашна задача 3 - Имплементација на ECB и CBC модови и „Исечи па залепи“ напад кај двата мода. 
Милена Кукољ 186085 

## Имплементација на ECB и CBC модовите

In [5]:
from des import DesKey

In [6]:
key0 = DesKey(b"some key")                  # for DES


Во оваа библиотека има една класа DesKey. Откако ќе се креира клучот, кој мора да биде во бајти и со должина од 8, 16 или 24 бајти. Од должината на клучот зависи и кој алгоритам ќе се користи понатаму. Доколку е 16 или 24 бајти, се користи 3DES алгоритамот. Кој алгоритам е искористен можеме да провериме со методот *is_single*.

In [7]:
key0.is_single() 

True

In [10]:
key0.encrypt(b"any long message") 

b"\x14\xfa\xc2 '\x00{\xa9\xdc;\x9dq\xcbr\x87Q"

Ако во методот за енкриптирање на порака не внесеме вредост за иницијалниот вектор, ќе се употреби модот CBC, како на примерот погоре. Во спротивно, ќе се употреби ECB модот на енкрипција. 

In [11]:
key0.encrypt(b"any long message", initial=0) 

b"\x14\xfa\xc2 '\x00{\xa9\xb2\xa5\xa7\xfb#\x86\xc5\x9b"

Оваа имплементација има и опција за додавање на дополнителни битови на крај од пораката доколку е со должина на бајти која не е делива со 8.

In [14]:
key0.encrypt(b"abc", padding=True) 

b'%\xd1KU\x8b_A\xa6'

In [15]:
key0.decrypt(b"%\xd1KU\x8b_A\xa6", padding=True) 

b'abc'

Откако ќе се повика методот *encrypt* тој ја повикува функцијата *handle* каде се одредува и кој метод ќе се користи за енкрипцијата, според вредноста на променливата *initial*. 

In [8]:
def handle(message, key, initial, padding, encryption):
    message = guard_message(message, padding, encryption)
    initial = guard_initial(initial)

    blocks = (struct.unpack(">Q", message[i: i + 8])[0] for i in iter_range(0, len(message), 8))

    if initial is None:
        # ECB
        encoded_blocks = ecb(blocks, key, encryption)
    else:
        # CBC
        encoded_blocks = cbc(blocks, key, initial, encryption)

    ret = b"".join(struct.pack(">Q", block) for block in encoded_blocks)
    return ret[:-ord(ret[-1:])] if not encryption and padding else ret

### ECB

In [1]:
def ecb(blocks, key, encryption):
    for block in blocks:
        yield encode(block, key, encryption)

In [2]:
def encode(block, key, encryption):
    for k in key:
        block = encode_block(block, k, encryption)
        encryption = not encryption

    return block

In [3]:
def encode_block(block, derived_keys, encryption):
    block = permute(block, 64, INITIAL_PERMUTATION)
    block = block >> 32, block & 0xffffffff

    if not encryption:
        derived_keys = reversed(derived_keys)
    for key in derived_keys:
        block = block[1], block[0] ^ f(block[1], key)

    return permute(block[1] << 32 | block[0], 64, INVERSE_PERMUTATION)

### CBC

In [4]:
def cbc(blocks, key, initial, encryption):
    if encryption:
        for block in blocks:
            initial = encode(block ^ initial, key, encryption)
            yield initial
    else:
        for block in blocks:
            initial, block = block, initial ^ encode(block, key, encryption)
            yield block

Всушност главната разлика во модовите на екрипција е дека во CBC блокот вредноста што се праќа на методот *encode* е резултатот од XOR операцијата на моменталниот блок и променливата *initial*. Променливата *initial* првично ја има вредноста на иницијалниот вектор, но потоа и ја доделуваме вредноста од енкриптираниот блок.

## „Исечи па залепи“ напад кај двата мода. 